### Code to merge datasets into a single one

In [91]:
import os
import cv2
from PIL import Image

In [92]:
iris_path = "../../data/casia_iris"
fingerprint_path = "../../data/casia_fingerprint"
faces_path = "../../data/celebfaces"

In [93]:
def get_n_subjects_face():
    with open(faces_path + "/identities.txt", "r") as f:
        text = f.read()
        
        lines = text.split("\n")
        ids = []
        for l in lines:
            if len(l) == 0:
                continue
            id_ = l.split(" ")[1]
            ids.append(id_)
        
        
        f.close()
        
        return len(set(ids))

In [94]:
iris_n_sub = len(os.listdir(iris_path))
iris_n_sub

1000

In [95]:
fingerprint_n_sub = len(os.listdir(fingerprint_path))
fingerprint_n_sub

500

In [96]:
face_n_sub = get_n_subjects_face()
face_n_sub

10177

In [97]:
def get_face_subs(n_sub):
    """Returns a list of tuples with the subject id and the image_names of the subject"""
    with open(faces_path + "/identities.txt", "r") as f:
        text = f.read()
        
        sub_data = {}
        lines = text.split("\n")
        
        for l in lines:
            if len(l) == 0:
                continue
            id_ = l.split(" ")[1]
            img = l.split(" ")[0]
            
            if id_ not in sub_data:
                sub_data[id_] = []
            sub_data[id_].append(img)
            
    subjects = list(sub_data.keys())
    subjects = subjects[:n_sub]
    
    return [(s, sub_data[s]) for s in subjects]

In [98]:
N_SUB = 500

iris_subs = os.listdir(iris_path)[:N_SUB]
fingerprint_subs = os.listdir(fingerprint_path)[:N_SUB]
face_subs = get_face_subs(N_SUB)    # (subject_id, [image_names])

In [99]:
face_sub_names = [s[0] for s in face_subs]

new_dataset_path = "../../data/combined_dataset"
if not os.path.exists(new_dataset_path):
    os.makedirs(new_dataset_path)


In [100]:
def create_dirs(new_sub_path):
    os.makedirs(new_sub_path, exist_ok=True)
    os.makedirs(new_sub_path + "/iris", exist_ok=True)
    os.makedirs(new_sub_path + "/fingerprint", exist_ok=True)
    os.makedirs(new_sub_path + "/face", exist_ok=True)

In [120]:
def get_iris_imgs(iris_sub):
    iris_sub_path = iris_path + "/" + iris_sub
    
    # Left imgs
    left_iris_imgs = os.listdir(iris_sub_path + "/L")
    left_iris_imgs = [Image.open(iris_sub_path + "/L/" + img) for img in left_iris_imgs if img.endswith(".jpg")]
    
    # Right imgs
    right_iris_imgs = os.listdir(iris_sub_path + "/R")
    right_iris_imgs = [Image.open(iris_sub_path + "/R/" + img) for img in right_iris_imgs if img.endswith(".jpg")]
    
    iris_data = left_iris_imgs + right_iris_imgs
    
    return iris_data
    
def get_fingerprint_imgs(fingerprint_sub):
    fingerprint_sub_path = fingerprint_path + "/" + fingerprint_sub
    
    # Left imgs
    left_fingerprint_imgs = os.listdir(fingerprint_sub_path + "/L")
    left_fingerprint_imgs = [Image.open(fingerprint_sub_path + "/L/" + img).convert("RGB") for img in left_fingerprint_imgs if img.endswith(".bmp")]
    
    # Right imgs
    right_fingerprint_imgs = os.listdir(fingerprint_sub_path + "/R")
    right_fingerprint_imgs = [Image.open(fingerprint_sub_path + "/R/" + img).convert("RGB") for img in right_fingerprint_imgs if img.endswith(".bmp")]
    
    fingerprint_data = left_fingerprint_imgs + right_fingerprint_imgs
    
    return fingerprint_data
    
def get_face_imgs(face_image_names: list):
    face_data = [Image.open(faces_path + "/faces/" + img) for img in os.listdir(faces_path + "/faces/") if img in face_image_names]
    return face_data


In [124]:
def write_imgs(imgs, new_sub_path):
    for i, img in enumerate(imgs):
        img.save(new_sub_path + "/{}.jpg".format(i), "JPEG")

In [127]:
for new_sub_id, (iris_sub, fingerprint_sub, face_pair )in enumerate(zip(iris_subs, fingerprint_subs, face_subs)):
    print(new_sub_id)
    
    # For each subject, create a folder with a new subject id
    new_sub_path = new_dataset_path + "/" + str(new_sub_id)

    create_dirs(new_sub_path)
    
    iris_imgs = get_iris_imgs(iris_sub)
    fingerprint_imgs = get_fingerprint_imgs(fingerprint_sub)
    face_imgs = get_face_imgs(face_pair[1])    
    
    write_imgs(iris_imgs, new_sub_path + "/iris")
    write_imgs(fingerprint_imgs, new_sub_path + "/fingerprint")
    write_imgs(face_imgs, new_sub_path + "/face")
    

0
1
2


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
